# Index Wikipedia into Solr

* Generate links like these: https://en.wikipedia.org/?curid=18630637 to avoid needing to canonicalize page names
* Assumes Wikipedia has been pre-processed into delta tables as per the notebook [wikipedia_gis_analysis_with_h3_and_deckgl.ipynb](wikipedia_gis_analysis_with_h3_and_deckgl.ipynb)

In [1]:
required_packages = {"pysolr","boltons"}
import pkg_resources
for lib in required_packages - {pkg.key for pkg in pkg_resources.working_set}:
    print(f"installing {lib}")
    %pip install -q --upgrade pip
    %pip install -q $lib
    pkg_resources.require(lib)


In [3]:
import os
os.environ['SOLR_HOST'] = 'localhost'

In [5]:
import os
solr_host       = os.environ.get('SOLR_HOST'      ,'solr')
solr_collection = os.environ.get('SOLR_COLLECTION','wiki')

In [6]:
import pysolr
solr = pysolr.Solr(f'http://{solr_host}:8983/solr/{solr_collection}', always_commit=True)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":0,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"-3"}},\n  "status":"OK"}\n'

In [7]:
if not "spark" in locals():
    import pyspark
    MAX_MEMORY = "8g"  # 24 gives OOM here. # 6 gives "out of heap space"
    spark = (pyspark.sql.SparkSession.builder.appName("MyApp") 
        .config("spark.jars.packages"            , "io.delta:delta-core_2.12:1.0.0") 
        .config("spark.sql.extensions"           , "io.delta.sql.DeltaSparkSessionExtension") 
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
        .config("spark.executor.memory"          , MAX_MEMORY) 
        .config("spark.driver.memory"            , MAX_MEMORY) 
        .config("spark.python.worker.reuse"      , False)
        .config("spark.task.maxFailures"         , 5)
        .enableHiveSupport() 
        .getOrCreate()        
        )
spark

In [8]:
spark.sql('select count(*),count(distinct title) from  wikipedia_silver_structured_templates').collect()

[Row(count(1)=21108360, count(DISTINCT title)=21108326)]

In [ ]:
spark.sql('select * from  wikipedia_silver_structured_templates limit 2').printSchema()

In [ ]:
s = """select title,infoboxes from  wikipedia_silver_structured_templates where size(infoboxes)>0 limit 20"""
print(json.dumps(spark.sql(s).take(1)[0].asDict(True),indent=1))

In [13]:
solr_url = f'http://{solr_host}:8983/solr/{solr_collection}'
from boltons.iterutils import remap
import datetime

class SolrForeachWriter:
    def open(self, partition_id, epoch_id):
        self._partition_id = partition_id
        self._epoch_id     = epoch_id
        self._batch_size   = 10000
        self._solr_url     = solr_url
        self.pending_docs  = []
        return True
        
    def insert_pending_docs(self):
        try:
            if self.pending_docs == []: return
            solr = pysolr.Solr(self._solr_url)
            solr.add(self.pending_docs)
            solr.commit()
            self.pending_docs = []
        except Exception as e:
            errmsg = f"{str(e)} adding {self.pending_docs}"
            raise
            
    def process(self,row):
        def map_to_solr_types(path,key,value):
            if value is None:
                return False
            if isinstance(value,datetime.datetime):
                return key,value.isoformat()+"Z"
            return key,value
        solr_data = remap(row.asDict(),visit=map_to_solr_types)
        self.pending_docs.append(solr_data)
        if len(self.pending_docs) >= self._batch_size:
            self.insert_pending_docs()
        return True
    
    def close(self,error):
        self.insert_pending_docs()
        return True

In [14]:

#spark.sql("""DROP TABLE IF EXISTS wikipedia_for_solr""")
spark.sql("""
          CREATE TABLE IF NOT EXISTS wikipedia_for_solr (
              id           string,
              title_txt_en string,
              body_txt_en  array<string>,
              person_s     array<string>,
              h3_1_s       array<string>,
              h3_2_s       array<string>,
              tags         array<string>
          ) USING DELTA
""")
# df = spark.createDataFrame([],"""
#               id           string,
#               title_txt_en string,
#               body_txt_en  array<string>,
#               person_s     array<string>,
#               h3_1_s       array<string>,
#               h3_2_s       array<string>,
#               tags         array<string>
# """)
# (df.write
#    .format('delta')
#    .mode('overwrite')
#    .option("mergeSchema", "true")
#    .saveAsTable('try_3_wiki_solr', format='delta', mode='overwrite', path='file:/home/jovyan/work/wikipedia_in_spark/notebooks/spark-warehouse/tables/try_3_wiki_solr')
#    #.saveAsTable('test_table', format='parquet', mode='overwrite', path='s3a://bucket/foo')
#    #.save('./tables/try_2_wikipedia_for_solr')
# )


DataFrame[]

In [15]:
#spark.sql("delete from wikipedia_for_solr")

Cute workaround for MERGE requiring matching columns...

*  https://mungingdata.com/pyspark/union-unionbyname-merge-dataframes/


Dedup to avoid

Py4JJavaError: An error occurred while calling o43.sql.
: java.lang.UnsupportedOperationException: Cannot perform Merge as multiple source rows matched and attempted to modify the same
target row in the Delta table in possibly conflicting ways. By SQL semantics of Merge,
when multiple source rows match on the same target row, the result may be ambiguous
as it is unclear which source row should be used to update or delete the matching
target row. You can preprocess the source table to eliminate the possibility of
multiple matches. Please refer to
https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-mergePy4JJavaError: An error occurred while calling o43.sql.
: java.lang.UnsupportedOperationException: Cannot perform Merge as multiple source rows matched and attempted to modify the same
target row in the Delta table in possibly conflicting ways. By SQL semantics of Merge,
when multiple source rows match on the same target row, the result may be ambiguous
as it is unclear which source row should be used to update or delete the matching
target row. You can preprocess the source table to eliminate the possibility of
multiple matches. Please refer to
https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge


https://docs.delta.io/latest/delta-update.html#-write-change-data-into-a-delta-table


In [23]:
import pyspark.sql.functions as psf

s = """
    SELECT
       base64(sha2(title, 224))   as id,
       title                             as title_txt_en,
       --array(body)                     as body_txt_en,
       array('')                         as person_s
    FROM  wikipedia_silver_structured_templates
    where lower(title) like 'a%' limit 1000000
"""
new_rows = (spark.sql(s)
            .selectExpr("id", "struct(title_txt_en,person_s) as other_cols")
            .groupBy("id")
            .agg(psf.max("other_cols").alias("latest"))
            .selectExpr("id", "latest.*")
           )
new_rows = spark.sql('select * from wikipedia_for_solr limit 0').unionByName(new_rows,allowMissingColumns=True)
new_rows.show()
new_rows.createOrReplaceTempView('new_rows')
new_rows.count()

+--------------------+--------------------+-----------+--------+------+------+----+
|                  id|        title_txt_en|body_txt_en|person_s|h3_1_s|h3_2_s|tags|
+--------------------+--------------------+-----------+--------+------+------+----+
|+++2+g44nLlXebhn+...|              Abreus|       null|      []|  null|  null|null|
|+++JdSu/cQmVT8S9O...|   About a Rib Chute|       null|      []|  null|  null|null|
|+++bKtQ0tXJ0s4anJ...|Acontista cayenne...|       null|      []|  null|  null|null|
|+++f0wez3y5KaqdMo...|         Androgynism|       null|      []|  null|  null|null|
|+++ij94QGgnl5gc3F...|APRA Music Awards...|       null|      []|  null|  null|null|
|++/4JcU4rSaPUVsMN...|Abdulla Fareed Al...|       null|      []|  null|  null|null|
|++/O5H5Rd2/J/DoXD...|      ATI EGA Wonder|       null|      []|  null|  null|null|
|++0+Jm6EVMNCq0e9k...|Amudala Palle, Pr...|       null|      []|  null|  null|null|
|++0G+ZMipwJOpHVJt...|      Ahrayut Leumit|       null|      []|  null|  nul

999998

In [24]:
for active_stream in spark.streams.active:
    print(f"Stream: {active_stream.name} - {active_stream.explain()}")
    print(dir(active_stream))
    #active_stream.stop()
    

In [25]:
solr.search(
    **{'q':'title_txt_en:dog','fl':'title_txt_en'}
           ).raw_response



{'responseHeader': {'status': 0,
  'QTime': 61,
  'params': {'q': 'title_txt_en:dog', 'fl': 'title_txt_en', 'wt': 'json'}},
 'response': {'numFound': 9096,
  'start': 0,
  'numFoundExact': True,
  'docs': [{'title_txt_en': 'Dogs Is Dogs'},
   {'title_txt_en': 'Dogs is Dogs'},
   {'title_txt_en': 'The Dogs'},
   {'title_txt_en': 'There is no dog'},
   {'title_txt_en': 'Dog'},
   {'title_txt_en': 'DOG'},
   {'title_txt_en': 'Dogs'},
   {'title_txt_en': 'That Dog'},
   {'title_txt_en': 'That Dog.'},
   {'title_txt_en': 'Dogging'}]}}

In [26]:
spark.sql("select count(*) from wikipedia_for_solr").show()

+--------+
|count(1)|
+--------+
| 1000000|
+--------+



In [27]:
#spark.sql("describe extended wikipedia_for_solr").show()

In [28]:
#spark.sql("select size(body_txt_en) from new_rows limit(1)").show()

In [29]:
spark.sql("""
    MERGE INTO wikipedia_for_solr
    USING new_rows
      ON new_rows.id = wikipedia_for_solr.id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
""")


DataFrame[]

In [30]:
spark.sql("select * from wikipedia_for_solr").count()

1000000

In [31]:
sdf = spark.readStream.format("delta").option("ignoreChanges",True).table("wikipedia_for_solr")
#df = spark.readStream.format("delta").option("ignoreChanges",True).load('./tables/try_2_wikipedia_for_solr')


In [32]:
sdf.isStreaming

True

In [33]:
!rm /tmp/new_rows_checkpoint4

rm: cannot remove '/tmp/new_rows_checkpoint4': Is a directory


In [34]:
sq = sdf.writeStream \
  .trigger(once=True) \
  .foreach(SolrForeachWriter()) \
  .option('checkpointLocation','/tmp/new_rows_checkpoint4') \
  .outputMode("update") \
  .start()



In [35]:
import time
t0 = time.perf_counter()
while True:
    astreams = spark.streams.active 
    if len(astreams)>0:
        print(f"At {time.perf_counter() - t0} stream: {astreams[0].name} - {astreams[0].explain()}")
        time.sleep(10)
    else:
        print(f"Done at {time.perf_counter() - t0}")
        break

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@4713031f
+- FileScan parquet [id#3777,title_txt_en#3778,body_txt_en#3779,person_s#3780,h3_1_s#3781,h3_2_s#3782,tags#3783] Batched: false, DataFilters: [], Format: Parquet, Location: TahoeBatchFileIndex[file:/home/jovyan/work/wikipedia_in_spark/notebooks/spark-warehouse/wikipedia..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:string,title_txt_en:string,body_txt_en:array<string>,person_s:array<string>,h3_1_s:arra...


At 0.008057176135480404 stream: None - None
== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@4713031f
+- FileScan parquet [id#3777,title_txt_en#3778,body_txt_en#3779,person_s#3780,h3_1_s#3781,h3_2_s#3782,tags#3783] Batched: false, DataFilters: [], Format: Parquet, Location: TahoeBatchFileIndex[file:/home/jovyan/work/wikipedia_in_spark/notebooks/spark-warehouse/wikipedia..., PartitionFilters: [], 

### Verify that it worked

In [36]:
solr.search(
    **{'q':'timestamp:[NOW-1HOUR TO NOW]','fl':'timestamp,title_txt_en,body_txt_en'}
).raw_response

{'responseHeader': {'status': 0,
  'QTime': 38,
  'params': {'q': 'timestamp:[NOW-1HOUR TO NOW]',
   'fl': 'timestamp,title_txt_en,body_txt_en',
   'wt': 'json'}},
 'response': {'numFound': 999998,
  'start': 0,
  'numFoundExact': True,
  'docs': [{'title_txt_en': 'Arthur Napoleon Raymond Robinson (airport)',
    'timestamp': '2021-06-07T10:01:59.506Z'},
   {'title_txt_en': 'Arts Council~Haliburton Highlands',
    'timestamp': '2021-06-07T10:01:59.506Z'},
   {'title_txt_en': 'Asago', 'timestamp': '2021-06-07T10:01:59.506Z'},
   {'title_txt_en': 'Appropriation Act 2007',
    'timestamp': '2021-06-07T10:01:59.209Z'},
   {'title_txt_en': 'Analog Gut', 'timestamp': '2021-06-07T10:01:59.506Z'},
   {'title_txt_en': 'Anthidium deceptum',
    'timestamp': '2021-06-07T10:01:59.209Z'},
   {'title_txt_en': 'Assyrian war of independence',
    'timestamp': '2021-06-07T10:01:59.506Z'},
   {'title_txt_en': 'Attar railway station',
    'timestamp': '2021-06-07T10:01:59.209Z'},
   {'title_txt_en': 'Agr

In [37]:
import requests
solr_host='192.168.12.110'
solr_collection='wiki'
solr_facet_query = {
    "query": "*:*",
    "limit": 0,
    "facet": {
        "docs_per_minute": {
            "type":  "range",
            "field": "timestamp",
            "start": "NOW/HOUR-10MINUTES",
            "end":   "NOW",
            "gap":   "+1SECOND",
            "other": "all"
        }
    }
}
headers = {'Content-type':'application/json'}
resp = requests.post(f"""http://{solr_host}:8983/solr/{solr_collection}/select?q=*:*""",
                 json = solr_facet_query,
                 headers = headers)
[b for b in resp.json()['facets']['docs_per_minute']['buckets'] if b['count']> 0]

[{'val': '2021-06-07T10:01:59Z', 'count': 159999},
 {'val': '2021-06-07T10:02:03Z', 'count': 159999},
 {'val': '2021-06-07T10:02:06Z', 'count': 144267},
 {'val': '2021-06-07T10:02:07Z', 'count': 10000},
 {'val': '2021-06-07T10:02:10Z', 'count': 150000},
 {'val': '2021-06-07T10:02:12Z', 'count': 150000},
 {'val': '2021-06-07T10:02:14Z', 'count': 10000},
 {'val': '2021-06-07T10:02:15Z', 'count': 140000},
 {'val': '2021-06-07T10:02:17Z', 'count': 75733}]


### Notes

* With "io.delta:delta-core_2.12:0.8.0", the steps abovegave errors like
  `AnalysisException: Table default.wikipedia_for_solr does not support either micro-batch or continuous scan.;`

* A workaround was to create both the streaming sources and the table merging into them like this: 
`.saveAsTable('try_3_wiki_solr', format='delta', mode='overwrite', path='file:/home/jovyan/work/wikipedia_in_spark/notebooks/spark-warehouse/tables/try_3_wiki_solr')`
  
  
  
  
### workaround this: 

* https://github.com/delta-io/delta/issues/594
* https://stackoverflow.com/questions/66106096/delta-lake-insert-into-sql-in-pyspark-is-failing-with-java-lang-nosuchmethoder
